In [4]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer
import pickle
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MultiLabelBinarizer




# 1. Data Understanding and Cleaning


In [5]:
import pandas as pd
data = pd.read_csv("D:\project_4\swiggy.csv")
data.head()

<>:2: SyntaxWarning: invalid escape sequence '\p'
<>:2: SyntaxWarning: invalid escape sequence '\p'
C:\Users\dell\AppData\Local\Temp\ipykernel_21856\1647844848.py:2: SyntaxWarning: invalid escape sequence '\p'
  data = pd.read_csv("D:\project_4\swiggy.csv")


,id,name,city,rating,rating_count,cost,cuisine,lic_no,link,address,menu
0,567335,AB FOODS POINT,Abohar,--,Too Few Ratings,₹ 200,"Beverages,Pizzas",22122652000138,https://www.swiggy.com/restaurants/ab-foods-po...,"AB FOODS POINT, NEAR RISHI NARANG DENTAL CLINI...",Menu/567335.json
1,531342,Janta Sweet House,Abohar,4.4,50+ ratings,₹ 200,"Sweets,Bakery",12117201000112,https://www.swiggy.com/restaurants/janta-sweet...,"Janta Sweet House, Bazar No.9, Circullar Road,...",Menu/531342.json
2,158203,theka coffee desi,Abohar,3.8,100+ ratings,₹ 100,Beverages,22121652000190,https://www.swiggy.com/restaurants/theka-coffe...,"theka coffee desi, sahtiya sadan road city",Menu/158203.json
3,187912,Singh Hut,Abohar,3.7,20+ ratings,₹ 250,"Fast Food,Indian",22119652000167,https://www.swiggy.com/restaurants/singh-hut-n...,"Singh Hut, CIRCULAR ROAD NEAR NEHRU PARK ABOHAR",Menu/187912.json
4,543530,GRILL MASTERS,Abohar,--,Too Few Ratings,₹ 250,"Italian-American,Fast Food",12122201000053,https://www.swiggy.com/restaurants/grill-maste...,"GRILL MASTERS, ADA Heights, Abohar - Hanumanga...",Menu/543530.json


In [6]:
data.shape

(148541, 11)

In [7]:
data.columns

Index(['id', 'name', 'city', 'rating', 'rating_count', 'cost', 'cuisine',
       'lic_no', 'link', 'address', 'menu'],
      dtype='object')

In [8]:
data.dtypes

id               int64
name            object
city            object
rating          object
rating_count    object
cost            object
cuisine         object
lic_no          object
link            object
address         object
menu            object
dtype: object

In [9]:
data.isnull().sum()

id                0
name             86
city              0
rating           86
rating_count     86
cost            131
cuisine          99
lic_no          229
link              0
address          86
menu              0
dtype: int64

In [10]:
null_percent = data.isnull().mean() * 100


print("Null Value Percentage (All Columns):")
print(null_percent.sort_values(ascending=False))

Null Value Percentage (All Columns):
lic_no          0.154166
cost            0.088191
cuisine         0.066648
rating          0.057896
address         0.057896
name            0.057896
rating_count    0.057896
city            0.000000
id              0.000000
link            0.000000
menu            0.000000
dtype: float64


Though lic_no column has more null values, may be this lic_no column is  not useful for recommendations.

dropping the column



In [11]:
data[data['name'].isnull()]


,id,name,city,rating,rating_count,cost,cuisine,lic_no,link,address,menu
31044,397961,NaN,"Greater Mohali,Chandigarh",NaN,NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/5-tara-grea...,NaN,Menu/397961.json
32912,308071,NaN,"West Chd,Chandigarh",NaN,NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/food-under-...,NaN,Menu/308071.json
33046,308662,NaN,"West Chd,Chandigarh",NaN,NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/franks-frie...,NaN,Menu/308662.json
33488,170889,NaN,"South Chd,Chandigarh",NaN,NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/dhaba-7-pre...,NaN,Menu/170889.json
36492,390394,NaN,"George Town,Chennai",NaN,NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/frozen-cafe...,NaN,Menu/390394.json
...,...,...,...,...,...,...,...,...,...,...,...
137122,256827,NaN,Satara,NaN,NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/rajdhani-pa...,NaN,Menu/256827.json
137546,567595,NaN,Shillong,NaN,NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/the-blue-ri...,NaN,Menu/567595.json
137614,116615,NaN,Shivamogga,NaN,NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/bakers-bowl...,NaN,Menu/116615.json
138380,401760,NaN,Sirsa,NaN,NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/murga-dlf-p...,NaN,Menu/401760.json


In [12]:
data.duplicated().sum()

np.int64(0)

Dropping the lic_no column

In [13]:
data.drop(columns=['lic_no'], inplace=True)


In [14]:
# To check the unquie value for rating cloumn
data["rating"].unique()

array(['--', '4.4', '3.8', '3.7', '3.6', '4.0', '4.2', '4.7', '4.1',
       '3.5', '3.1', '3.4', '3.3', '4.8', '3.9', '2.7', '4.3', '2.9',
       '4.5', '2.5', '3.2', '2.4', '4.6', '3.0', '2.8', '2.3', '5.0',
       '2.6', '2.2', '1.4', '1.9', '4.9', '2.1', '1.3', '2.0', '1.8',
       '1.6', '1.1', '1.5', nan, '1.0', '1.2', '1.7'], dtype=object)

 **FOUND SOME NAN VALUES AND '--' IN THE COLUMN**

In [15]:
# To check the unqiue values for rating_count cloumn
data["rating_count"].unique()

array(['Too Few Ratings', '50+ ratings', '100+ ratings', '20+ ratings',
       '500+ ratings', '1K+ ratings', '5K+ ratings', nan, '10K+ ratings'],
      dtype=object)

In [16]:
data["rating_count"].value_counts()

rating_count
Too Few Ratings    87014
20+ ratings        21636
100+ ratings       20548
50+ ratings        12009
500+ ratings        4396
1K+ ratings         2739
5K+ ratings           98
10K+ ratings          15
Name: count, dtype: int64

**WE HAVE TO REMOVE + k SYMBOL AND CONVERT IN NORMAL FORMAT**

In [17]:
# To check the values for cost cloumns
data["cost"].unique()

array(['₹ 200', '₹ 100', '₹ 250', '₹ 150', '₹ 300', '₹ 700', '₹ 650',
       '₹ 400', '₹ 350', '₹ 60', '₹ 110', '₹ 399', '₹ 249', '₹ 500',
       '₹ 99', '₹ 120', '₹ 499', '₹ 299', '₹ 199', '₹ 50', '₹ 180',
       '₹ 349', '₹ 1000', '₹ 599', '₹ 600', '₹ 800', '₹ 450', '₹ 149',
       '₹ 290', '₹ 175', '₹ 125', '₹ 8', '₹ 375', '₹ 275', '₹ 425',
       '₹ 225', '₹ 325', '₹ 75', '₹ 160', '₹ 550', '₹ 220', '₹ 20',
       '₹ 59', '₹ 1200', '₹ 210', '₹ 30', '₹ 310', '₹ 70', '₹ 170',
       '₹ 449', '₹ 280', '₹ 320', '₹ 1300', '₹ 850', '₹ 900', '₹ 40',
       '₹ 1500', '₹ 140', '₹ 1100', '₹ 410', '₹ 80', '₹ 10', '₹ 1245',
       '₹ 510', '₹ 90', '₹ 260', '₹ 1800', '₹ 5', '₹ 240', '₹ 460',
       '₹ 1900', '₹ 352', '₹ 298', '₹ 2', '₹ 252', '₹ 330', '₹ 750',
       '₹ 130', '₹ 2000', '₹ 198', '₹ 230', '₹ 999', '₹ 3999', nan,
       '₹ 235', '₹ 1600', '₹ 55', '₹ 179', '₹ 129', '₹ 360', '₹ 85',
       '₹ 248', '₹ 270', '₹ 25', '₹ 159', '₹ 370', '₹ 1050', '₹ 49',
       '₹ 699', '₹ 340', '₹ 190', 

In [18]:
data["cost"].value_counts()

cost
₹ 200    38635
₹ 300    29701
₹ 250    19745
₹ 150    12096
₹ 400    11711
         ...  
₹ 216        1
₹ 88         1
₹ 101        1
₹ 312        1
₹ 64         1
Name: count, Length: 363, dtype: int64

**WE HAVE TO REMOVE RUPEES SYMBOL**

In [19]:
#Fom rating cloumn replace "--" to nan for purpouse of converting to Numeric
data["rating"]= data["rating"].replace('--','4.0')

#Fom rating_count cloumn replace 'Too Few Ratings' to '25' for purpouse of converting to Numeric
data["rating_count"] = data["rating_count"].replace('Too Few Ratings','25')

# from rating_count cloumn we can need only the number so we can extract the number based on index basics
data["rating_count"] = data["rating_count"].astype(str).str[:3]

#After getting the extracting numnber in rating_count cloumn we have "+" symbol so we can replace with ""(space)
data["rating_count"] = data["rating_count"].str.replace("+","")

# Replace "k" to "000" in the rating_count cloumn
data["rating_count"] = data["rating_count"].str.replace("K","000")

#Replace "₹" to ""(space) in the cost cloumn
data["cost"] = data["cost"].str.replace("₹", "")

#To convert the rating cloumn object into float
data["rating"] = data["rating"].astype(float)

#To convert the rating_count cloumn object into float
data["rating_count"] = data["rating_count"].astype(float)

#To convert the cost cloumn object into float
data["cost"] = data["cost"].astype(float)

After converting the 3 cloumns into numerical We can check null values

In [20]:
# Calculate null values count
null_counts = data.isnull().sum()

# Calculate percentage of null values
null_percentage = (null_counts / len(data)) *100

# Combine both into a DataFrame
null_info = pd.DataFrame({'Null Count': null_counts, 'Null Percentage': null_percentage})

print(null_info)

              Null Count  Null Percentage
id                     0         0.000000
name                  86         0.057896
city                   0         0.000000
rating                86         0.057896
rating_count          86         0.057896
cost                 131         0.088191
cuisine               99         0.066648
link                   0         0.000000
address               86         0.057896
menu                   0         0.000000


In [21]:
# To check what are null values in rating cloumn
data[data["rating"].isnull()]

,id,name,city,rating,rating_count,cost,cuisine,link,address,menu
31044,397961,NaN,"Greater Mohali,Chandigarh",NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/5-tara-grea...,NaN,Menu/397961.json
32912,308071,NaN,"West Chd,Chandigarh",NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/food-under-...,NaN,Menu/308071.json
33046,308662,NaN,"West Chd,Chandigarh",NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/franks-frie...,NaN,Menu/308662.json
33488,170889,NaN,"South Chd,Chandigarh",NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/dhaba-7-pre...,NaN,Menu/170889.json
36492,390394,NaN,"George Town,Chennai",NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/frozen-cafe...,NaN,Menu/390394.json
...,...,...,...,...,...,...,...,...,...,...
137122,256827,NaN,Satara,NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/rajdhani-pa...,NaN,Menu/256827.json
137546,567595,NaN,Shillong,NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/the-blue-ri...,NaN,Menu/567595.json
137614,116615,NaN,Shivamogga,NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/bakers-bowl...,NaN,Menu/116615.json
138380,401760,NaN,Sirsa,NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/murga-dlf-p...,NaN,Menu/401760.json


In [22]:
# # To check what are null values in name cloumn
data[data["name"].isnull()]


,id,name,city,rating,rating_count,cost,cuisine,link,address,menu
31044,397961,NaN,"Greater Mohali,Chandigarh",NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/5-tara-grea...,NaN,Menu/397961.json
32912,308071,NaN,"West Chd,Chandigarh",NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/food-under-...,NaN,Menu/308071.json
33046,308662,NaN,"West Chd,Chandigarh",NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/franks-frie...,NaN,Menu/308662.json
33488,170889,NaN,"South Chd,Chandigarh",NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/dhaba-7-pre...,NaN,Menu/170889.json
36492,390394,NaN,"George Town,Chennai",NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/frozen-cafe...,NaN,Menu/390394.json
...,...,...,...,...,...,...,...,...,...,...
137122,256827,NaN,Satara,NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/rajdhani-pa...,NaN,Menu/256827.json
137546,567595,NaN,Shillong,NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/the-blue-ri...,NaN,Menu/567595.json
137614,116615,NaN,Shivamogga,NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/bakers-bowl...,NaN,Menu/116615.json
138380,401760,NaN,Sirsa,NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/murga-dlf-p...,NaN,Menu/401760.json


In [23]:
## To check what are null values in rating_count cloumn
data[data["rating_count"].isnull()]

,id,name,city,rating,rating_count,cost,cuisine,link,address,menu
31044,397961,NaN,"Greater Mohali,Chandigarh",NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/5-tara-grea...,NaN,Menu/397961.json
32912,308071,NaN,"West Chd,Chandigarh",NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/food-under-...,NaN,Menu/308071.json
33046,308662,NaN,"West Chd,Chandigarh",NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/franks-frie...,NaN,Menu/308662.json
33488,170889,NaN,"South Chd,Chandigarh",NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/dhaba-7-pre...,NaN,Menu/170889.json
36492,390394,NaN,"George Town,Chennai",NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/frozen-cafe...,NaN,Menu/390394.json
...,...,...,...,...,...,...,...,...,...,...
137122,256827,NaN,Satara,NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/rajdhani-pa...,NaN,Menu/256827.json
137546,567595,NaN,Shillong,NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/the-blue-ri...,NaN,Menu/567595.json
137614,116615,NaN,Shivamogga,NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/bakers-bowl...,NaN,Menu/116615.json
138380,401760,NaN,Sirsa,NaN,NaN,NaN,NaN,https://www.swiggy.com/restaurants/murga-dlf-p...,NaN,Menu/401760.json


IN THIS PROCESS NAME, RATING, RATING_COUNT,ADDRESS HAS SAME NUMBER OF NAN VALUES

NEED TO CHECK ANY RELATIONSHIP BETWEEN THESE COLUMNS

In [24]:
# First, check which rows have nulls in those 4 columns
null_rows = data[data[['name', 'rating', 'rating_count', 'address']].isnull().all(axis=1)]

# Count how many such rows there are
print(f"Rows where all 4 columns are null: {len(null_rows)}")


Rows where all 4 columns are null: 86


In [25]:
# See if these rows belong to a specific city
print(null_rows['city'].value_counts())

# Or maybe the 'cost' is also null in these rows?
print(null_rows['cost'].isnull().sum())


city
Kurukshetra            8
Jadavpur,Kolkata       4
Ichalkaranji           3
Gomti Nagar,Lucknow    3
Rohtak                 3
                      ..
Satara                 1
Shillong               1
Shivamogga             1
Sirsa                  1
Thiruvananthapuram     1
Name: count, Length: 61, dtype: int64
86


The missing values aren't tied to a specific city — so this is not a location-based issue.



In [26]:
df_check = data[['name', 'rating', 'rating_count', 'address']]
print((df_check.isnull().sum(axis=1) == 4).sum())


86


There is a clear relationship between these four columns in those rows—they always go missing together, and it's not due to city or cost. Most likely:

These rows are incomplete records (e.g., broken entries during scraping).

Could be restaurants that didn’t load fully or were placeholder entries.



dropping the missing rows

In [27]:
data = data[~data[['name', 'rating', 'rating_count', 'address']].isnull().all(axis=1)]


In [28]:
data.isnull().sum()

id               0
name             0
city             0
rating           0
rating_count     0
cost            45
cuisine         13
link             0
address          0
menu             0
dtype: int64

**COST, CUISINE HAS SOME NULL VALUES**

**IMPUTING THE NULL VALUES COLUM**

In [29]:
data['cost'] = data['cost'].fillna(data['cost'].median())


In [30]:
# Function to get the most common cuisine for a city
def fill_most_frequent_cuisine(group):
    mode = group['cuisine'].mode()
    if not mode.empty:
        group['cuisine'] = group['cuisine'].fillna(mode[0])
    else:
        group['cuisine'] = group['cuisine'].fillna('Unknown')  # fallback if no mode
    return group

# Apply it group by city
data = data.groupby('city').apply(fill_most_frequent_cuisine).reset_index(drop=True)


C:\Users\dell\AppData\Local\Temp\ipykernel_21856\3174056783.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = data.groupby('city').apply(fill_most_frequent_cuisine).reset_index(drop=True)


In [31]:
 data.isnull().sum()

id              0
name            0
city            0
rating          0
rating_count    0
cost            0
cuisine         0
link            0
address         0
menu            0
dtype: int64

In [32]:
data.shape

(148455, 10)

before cleaning process the shape of the data is (148541, 11) , now (148455, 10) , we have lost 86 rows and 1 cloumn


In [33]:
#To convert into csv file
data.to_csv("cleaned_data.csv", index=False)

# 2. Data Preprocessing

In [34]:
print(data['city'].sample(50))  # or just use head() if you want the first few rows


129970                   T Nagar,Chennai
75346                       Kothrud,Pune
145417              Whitefield,Bangalore
3845                     Aliganj,Lucknow
97220              Navrangpura,Ahmedabad
139052                      Vashi,Mumbai
105279              Panchkula,Chandigarh
130578                  Tambaram,Chennai
17029                           Bareilly
65460                 Kadugodi,Bangalore
67314                        Kanchrapara
108783                       Pondicherry
79270                          Lakhimpur
107240                  Perambur,Chennai
60164             Iyyapanthangal,Chennai
105971                             Patna
144325                        Wakad,Pune
119042                      Sadar,Nagpur
138442                      Vashi,Mumbai
38845          Electronic City,Bangalore
113942                  Rajarhat,Kolkata
88910                    Maunath-bhanjan
129601                     Swargate,Pune
141773                  Viman Nagar,Pune
124468          

It looks like your city column has a mix of city names and some regions or specific areas (e.g., Ramapuram, Chennai, Piplod, Surat, Kadubeesanahalli, Bangalore). There might also be inconsistent formatting, like commas separating the area and city name.



In [35]:
# Split into two columns: 'area' and 'city'
data[['area', 'city']] = data['city'].str.split(',', n=1, expand=True)

# Clean up whitespace
data['area'] = data['area'].str.strip().fillna('Unknown').str.lower()
data['city'] = data['city'].str.strip().fillna('Unknown').str.lower()


In [36]:
# Split the 'cuisine' string into a list and strip spaces from each item
data['cuisine'] = data['cuisine'].str.split(',').apply(lambda x: [i.strip() for i in x])


In [37]:
data.columns


Index(['id', 'name', 'city', 'rating', 'rating_count', 'cost', 'cuisine',
       'link', 'address', 'menu', 'area'],
      dtype='object')

In [38]:
data.dtypes

id                int64
name             object
city             object
rating          float64
rating_count    float64
cost            float64
cuisine          object
link             object
address          object
menu             object
area             object
dtype: object

In [39]:
print(data.columns.tolist())


['id', 'name', 'city', 'rating', 'rating_count', 'cost', 'cuisine', 'link', 'address', 'menu', 'area']


# encoding the categorical data

city -----	One-Hot Encoder	 ----  Limited categories, good for distance-based models like K-Means

name -----	Label Encoder -----	Each restaurant is unique, so one-hot would explode; label encoding preserves IDs

cuisine -----	MultiLabel Binarizer -----	Many entries contain multiple cuisines, separated by commas — so treat it like a multi-label field

In [40]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MultiLabelBinarizer
import pickle

# Load your cleaned data
data = pd.read_csv('cleaned_data.csv')

# ---------------------- Preprocessing ----------------------

# Ensure 'cuisine' is a list for each row
data['cuisine'] = data['cuisine'].apply(lambda x: [i.strip() for i in x.split(',')] if isinstance(x, str) else [])


# Label encode restaurant names
name_encoder = LabelEncoder()
data['name_encoded'] = name_encoder.fit_transform(data['name'])
data['name_encoded'] = data['name_encoded'].astype('int32')

# One-hot encode city
city_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
city_encoded = city_encoder.fit_transform(data[['city']])
city_encoded_df = pd.DataFrame(city_encoded, columns=city_encoder.get_feature_names_out(['city']))
city_encoded_df = city_encoded_df.astype('uint8')  # memory optimization

# Multi-label binarizer for cuisine
mlb = MultiLabelBinarizer()
cuisine_encoded = mlb.fit_transform(data['cuisine'])
cuisine_encoded_df = pd.DataFrame(cuisine_encoded, columns=mlb.classes_)
cuisine_encoded_df = cuisine_encoded_df.astype('uint8')  # memory optimization

# ---------------------- Combine All Encoded Features ----------------------

final_features = pd.concat([
    data[['name_encoded']].reset_index(drop=True),
    city_encoded_df.reset_index(drop=True),
    cuisine_encoded_df.reset_index(drop=True)
], axis=1)

# ---------------------- Save Encoded Data ----------------------

final_features.to_csv('encoded_data.csv', index=False)
print("✅ Encoded data saved to 'encoded_data.csv'.")

# Save encoders for inference
pickle.dump(name_encoder, open('name_encoder.pkl', 'wb'))
pickle.dump(city_encoder, open('city_encoder.pkl', 'wb'))
pickle.dump(mlb, open('cuisine_encoder.pkl', 'wb'))

print("✅ Encoders saved.")


✅ Encoded data saved to 'encoded_data.csv'.
✅ Encoders saved.


In [41]:
# --- Scale data ---
scaler = StandardScaler()
scaled_data = scaler.fit_transform(final_features)


# --- Apply PCA ---
pca= PCA(n_components=30)  # You can increase/decrease depending on variance explained
pca_data = pca.fit_transform(scaled_data)

# --- Train KMeans ---
kmeans = KMeans(n_clusters=10, random_state=42,n_init=10)  # you can tune n_clusters
kmeans.fit(pca_data)

# --- Save PCA-transformed data ---
pd.DataFrame(pca_data).to_csv("pca_encoded_data.csv", index=False)

# --- Save PCA model ---
with open("pca_model.pkl", "wb") as f:
    pickle.dump(pca, f)

# --- Save models ---
with open("kmeans_model.pkl", "wb") as f:
    pickle.dump(kmeans, f)

with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

print("✅ KMeans and Scaler saved successfully!")

✅ KMeans and Scaler saved successfully!


In [42]:
cluster_labels = kmeans.predict(pca_data)
data['cluster'] = cluster_labels


In [43]:
# ✅ Save input column order for matching later
with open("pca_input_columns.pkl", "wb") as f:
    pickle.dump(final_features.columns.tolist(), f)